In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv('/content/drive/MyDrive/Abc/Session Descriptions.csv')

In [3]:
df['name']=df['Description'].str.split(pat=':',expand=True).get(0)
df['Title']=df['Description'].str.split(pat=':',expand=True).get(1)
df=df.drop('Description',axis=1)

In [4]:
def find_d(x):
    if re.search("desktop",x.lower()):
        return "Desktop"
    elif re.search("server",x.lower()):
        return "Server"
    elif re.search("prep",x.lower()):
        return "Prep"
    else:
        return "Community"

In [5]:
df['Subject']=df['Title'].apply(lambda x: find_d(x))
df['Speaker']=df['name'].apply(lambda x:"".join([word[0] for word in x.split()]) )
df['Deduplication Flag']=df['Title'].apply(lambda x: True if re.search("deduplication",x.lower()) else False)

In [6]:
tc=df[['Session Number','Subject','Speaker','Deduplication Flag']][df['Deduplication Flag']==True]

In [7]:
tc

,Session Number,Subject,Speaker,Deduplication Flag
0,1,Server,AP,True
5,6,Prep,TP,True
10,11,Desktop,RV,True


#####End of output1

In [8]:
sf=pd.read_csv('/content/drive/MyDrive/Abc/preppin_w19/Rooms.csv').set_index('Room')
sf

,Floor 1,Floor 2,Floor 3
Room,,,
1,Servicing Server - AP on Server,Preparing Prep - TP on Prep,"Stats, stats, stats - RV on Desktop"
2,Preppin' Power - JM on Prep,Severe Server - JM on Server,Executive Track - TB on Community
3,Desktop Design - EB on Desktop,Fluffy Cloud - LK on Server,NaN
4,Commit to the Community - TP on Community,"Data, data everywhere but not able to analyse ...",NaN
5,Upping Performance - TN on Desktop,Awesome Analysis - RM on Desktop,NaN


In [9]:
t=sf.unstack().reset_index().dropna()
t.rename(columns={'level_0':'Floor', 0:'Description'},inplace=True)

In [10]:
t['Room']=t['Room'].astype('str')

In [11]:
t['Title']=t['Description'].str.split(pat='-',expand=True).get(1)
t['speaker']=t['Title'].str.split(pat='on ',expand=True).get(0)
t['subject']=t['Title'].str.split(pat='on ',expand=True).get(1)
t['Room Num']=t['Floor'].str.split(pat=' ',expand=True).get(1)+'0'+t['Room']

In [12]:
t['speaker']=t['speaker'].apply(lambda x:x.strip())

In [13]:
t=t[['speaker','subject','Room Num']]
t

,speaker,subject,Room Num
0,AP,Server,101
1,JM,Prep,102
2,EB,Desktop,103
3,TP,Community,104
4,TN,Desktop,105
5,TP,Prep,201
6,JM,Server,202
7,LK,Server,203
8,CA,Prep,204
9,RM,Desktop,205


In [14]:
tc2=tc.merge(t,how='inner',left_on=['Speaker','Subject'],right_on=['speaker','subject'])
tc2=tc2[tc2['Room Num']=='201']

In [15]:
tcf=tc2[['Speaker','Subject','Session Number','Room Num']]
tcf

,Speaker,Subject,Session Number,Room Num
1,TP,Prep,6,201


#####End of Output 2